# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [8]:
1.0 + 1.4142135623730951 + 1.7320508075688772 + 2.0 + 2.23606797749979

8.382332347441762

In [9]:
limit = 13
generator = square_root_generator(limit)

for val in generator:
    print(val)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [69]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# for person in people_1():
#     print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


# for person in people_2():
#     print(person)


In [72]:
import dlt
import duckdb

# configure the pipeline with your destination details
pipeline = dlt.pipeline(
    pipeline_name='load_person_data',
    destination='duckdb',
    dataset_name='person_data'
)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# data = list(people_1())
load_data = pipeline.run(people_1(), table_name='people')

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
conn.sql("show tables")

# conn.sql('TRUNCATE TABLE people_1')


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_2            │
└─────────────────────┘

In [74]:
conn.sql('SELECT * FROM people')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1709305910.5168567 │ gqiWmy1D+i4NSQ │
│     2 │ Person_2 │    27 │ City_A  │ 1709305910.5168567 │ 5k97Jjvm+rsvaw │
│     3 │ Person_3 │    28 │ City_A  │ 1709305910.5168567 │ zqvuL3IMijPLZw │
│     4 │ Person_4 │    29 │ City_A  │ 1709305910.5168567 │ KYK6lwBFqzmNlA │
│     5 │ Person_5 │    30 │ City_A  │ 1709305910.5168567 │ mg4w1s5MdVsAWQ │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘

In [75]:
load_data2 = pipeline.run(people_2(), table_name='people')

In [76]:
conn.sql('SELECT * FROM people')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1709305910.5168567 │ gqiWmy1D+i4NSQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1709305910.5168567 │ 5k97Jjvm+rsvaw │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1709305910.5168567 │ zqvuL3IMijPLZw │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1709305910.5168567 │ KYK6lwBFqzmNlA │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1709305910.5168567 │ mg4w1s5MdVsAWQ │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1709305938.9922628 │ Xy+fO4rAGFXZRA │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1709305938.9922628 │ hi/IvZ7sHV9RIg │ Job_4      │
│     5 │ 

In [77]:
conn.sql('SELECT SUM(age) FROM people')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
# #Install the dependencies
# %%capture
# !pip install dlt[duckdb]

In [78]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# data = list(people_1())
load_data = pipeline.run(people_1(), table_name='people_merge', primary_key='id')

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
conn.sql("show tables")

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_2            │
│ people_merge        │
└─────────────────────┘

In [79]:
conn.sql('SELECT * FROM people_merge')

┌───────┬──────────┬───────┬─────────┬───────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │   _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar      │    varchar     │
├───────┼──────────┼───────┼─────────┼───────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1709307268.867604 │ njCMkkUGW2EfRA │
│     2 │ Person_2 │    27 │ City_A  │ 1709307268.867604 │ A6NUBaQ8+bV1EA │
│     3 │ Person_3 │    28 │ City_A  │ 1709307268.867604 │ jobUgqxYmt3PcA │
│     4 │ Person_4 │    29 │ City_A  │ 1709307268.867604 │ UMDQXBIv9EZneA │
│     5 │ Person_5 │    30 │ City_A  │ 1709307268.867604 │ UgYATtpnT9OE6g │
└───────┴──────────┴───────┴─────────┴───────────────────┴────────────────┘

In [80]:
conn.sql('SELECT SUM(age) FROM people_merge')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [81]:
load_data2 = pipeline.run(people_2(), table_name='people_merge', write_disposition='merge')

In [82]:
conn.sql('SELECT * FROM people_merge')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1709307268.867604  │ njCMkkUGW2EfRA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1709307268.867604  │ A6NUBaQ8+bV1EA │ NULL       │
│     8 │ Person_8 │    38 │ City_B  │ 1709307349.4443488 │ yX97qTE+TNgJEA │ Job_8      │
│     4 │ Person_4 │    34 │ City_B  │ 1709307349.4443488 │ NfgC5seqtIww1A │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ 1709307349.4443488 │ ZZ4K+mcceFv8ow │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1709307349.4443488 │ zmxj5sJiSTIqyA │ Job_7      │
│     3 │ Person_3 │    33 │ City_B  │ 1709307349.4443488 │ M4Yf6XZFUILwDQ │ Job_3      │
│     6 │ 

In [83]:
conn.sql('SELECT SUM(age) FROM people_merge')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX